# Making a Script to Average the Colvars
    Want to replace the DHAM algorithm for generating the initial path of a string.
    Need to be relatively efficient, fast, and easy to run on Midway
    
### Assume that every file we need is in the current directory... And we're off

In [1]:
## import all of our fun stuff
%matplotlib inline
import pandas
import numpy as np
import matplotlib.pyplot as mp

In [16]:
# The bash commands above this called python script convert the *colvars.traj file to a *out file
x=pandas.read_csv('data0.out',header=None)
colvar=x.as_matrix(columns=None)

# Import the input colvars that are used as reference
y=pandas.read_csv('inp0.out',header=None)
input=y.as_matrix(columns=None)
# this current method removes the first line of colvar (which isn't great)

In [17]:
#Need to do that weird little thing where I make the [-180,180] range to [0,360]
# NOTE, WE ONLY PICKED COLVAR[2] and [4] because 
for i in range(0,len(colvar[:,2])):
        if colvar[i,2] < 0:
            colvar[i,2]=colvar[i,2]+360
        if colvar[i,5] < 0:
            colvar[i,5]=colvar[i,5]+360
input[1]=input[1]+360
input[4]=input[4]+360

In [18]:
# so we have, in order: R, Theta, Phi, Psi, theta_low, phi_low
# This can change from colvars to colvars though, so make sure you check...
end=len(colvar)
# Take all the averages, but throw out the first 500 data points so it's equilibrated
Ravg=np.average(colvar[500:end,0])
Theta_avg=np.average(colvar[500:end,1])
Phi_avg=np.average(colvar[500:end,2])
Psi_avg=np.average(colvar[500:end,3])
theta_l_avg=np.average(colvar[500:end,4])
phi_l_avg=np.average(colvar[500:end,5])

In [19]:
# Take all the stdevs
Rstd=np.std(colvar[500:end,0])
Theta_atd=np.std(colvar[500:end,1])
Phi_std=np.std(colvar[500:end,2])
Psi_std=np.std(colvar[500:end,3])
theta_l_std=np.std(colvar[500:end,4])
phi_l_std=np.std(colvar[500:end,5])

In [20]:
# Get our colvar inputs (order Theta, Phi, Psi, theta_low, phi_low, R)
theta=input[0]
phi=input[1]
psi=input[2]
theta_low=input[3]
phi_low=input[4]
R=input[5]

In [23]:
# Great now let's calculate our new values
# Want to weight the new angles based on the initial colvar

# Set some weight w for the averaging (this w will be a function of r at some point)
w=0.5 # w is between 0 and 1 always

theta_new=w*Theta_avg+(1-w)*theta
phi_new=w*Phi_avg+(1-w)*phi
psi_new=w*Psi_avg+(1-w)*psi
theta_l_new=w*theta_l_avg+(1-w)*theta_low
phi_l_new=w*phi_l_avg+(1-w)*phi_low
print('new_colvar','input_colvar','average_colvar')
print(theta_new,theta,Theta_avg)
print(phi_new,phi,Phi_avg)
print(psi_new,psi,Psi_avg)
print(theta_l_new,theta_low,theta_l_avg)
print(phi_l_new,phi_low,phi_l_avg)

new_colvar input_colvar average_colvar
[ 98.38090696] [ 101.] 95.7618139216
[ 193.3304726] [ 188.] 198.660945203
[ 26.85365503] [ 27.] 26.7073100576
[ 104.39108047] [ 102.] 106.782160937
[ 184.4790431] [ 190.] 178.958086209


In [24]:
# R new is going to be created also in a creative way hopefully, but for now let's
# just do a simple ∆R and let it be.

dR=0.5
R_new=R+dR

In [28]:
# Save these new beautiful variables
final=[theta_new,phi_new,psi_new,theta_l_new,phi_l_new,R_new]
# print 5 digits after the decimal place
np.savetxt('inp1.out',final,fmt='%.5e')